In [1]:
# Importation
import pandas as pd
import plotly.express as px

# **I - Data management**

In [2]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

## Présentation des données 


Le jeu de données qu'on traite dans le notebook est issue du jeu de données open source  [La fiscalité et les dépenses publiques](https://www.data.gouv.fr/fr/datasets/r/bc085888-e6bd-445d-b3f4-632190c29e3f). Celui-ci est notament constitué des réponses textuelles des contribuables à des questions portant sur la fiscalité en France. Dans cette étude, on se penche principalement sur la question suivante  : *Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?*.

Pour extraire idées majeures des réponses textuelles, on utilise le modèle de langue ollama : *hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4*. Avec les hyperparamètres **top_p = 0.95** et **temperature = 0**.

On donne à ce LLM le prompt suivant :

system = """
But: extraire les idées principales DISTINCTES d’un texte pour analyse.

Règles:
1. N’utiliser QUE le contenu entre <<< TEXT >>>.
2. Extraire la liste des idées DISTINCTES et PRINCIPALES.
   - Chaque idée = une phrase claire, autonome, reformulée si nécessaire.
3. Pour CHAQUE idée, annoter:
   - type: "statement" (constat) OU "proposition" (suggestion/recommandation/objectif).
   - syntax: "negative" si la phrase contient une négation explicite (ex.: "ne", "n'", "ne pas", "ne plus", "non"), sinon "positive".
   - semantic: "positive", "negative" ou "neutral" (valence sémantique).
4. Sortie STRICTEMENT en CSV avec entête EXACTE:
   CSV:description,type,syntax,semantic
   - Délimiteur: virgule.
   - Chaque description entre guillemets doubles.
   - Échapper tout guillemet interne par duplication (ex.: ""chat"").
   - NE RIEN AJOUTER d’autre (pas de texte avant/après, pas de code fences).
   - Pas de lignes vides.

Exemple:
CSV:description,type,syntax,semantic
"Les chats retombent sur leurs pattes",statement,positive,neutral
"Les chats n'ont pas neuf vies",statement,negative,negative
"Il faut mieux prendre soin des chats pour prolonger leur vie",proposition,positive,positive
"""
user = """<<< {input} >>>"""

Ainsi, on construit le jeu de données **df** contenant les variables :

- **authorId** : Identifiant de l’auteur.
- **contrib_index** : Numéro de la contribution.
- **contribution** : Texte original de l'auteur.
- **C** : Score calculé par la métrique *QualIT*.
- **n_ideas** : Nombre d’idées extraites.
- **len_contrib** : Longueur du texte original.
- **Matthias, Yannis, Garance** : Note allant de 0 à 10 évaluant selon la personne la qualité de l'extraction.
- **Hallucinations** : Présence d'hallucination dans l'extraction.
- **Idées_non_ind** : Idées dénuée de sens dans l'extraction.

On construit ensuite de nouvelles variables à partir de ces données brutes pour faciliter l'analyse :

In [3]:
# Calcul du score humain moyen, normalisé sur 1
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1) / 10

# Nombre de tokens par contribution et par extractions (comptage simple par espaces)
df["nb_tokens_contrib"] = df["contribution"].apply(lambda x: len(str(x).split()))
df["nb_tokens_extraction"] = df["ideas_text"].apply(lambda x: len(str(x).split()))

# Nombre de charactères par contribution et par extractions
df["nb_char_contrib"] = df["contribution"].apply(len)
df["nb_char_extraction"] = df["ideas_text"].apply(len)

# Hallucinations et Idées_non_ind en catégories
df['hal_cat'] = df['Hallucinations'].astype("category")
df["idees_cat"] = df["Idées_non_ind"].astype("category")

# Catégories de longueur
length_bins = [0, 250, 500, 2000, float("inf")]
length_labels = ["Très courte (0-250)", "Courte (251-500)", "Moyenne (501-2000)", "Longue (2000+)"]
df["contrib_cat"] = pd.cut(df["nb_char_contrib"], bins=length_bins, labels=length_labels)

# **II - Validité de la notation humaine**

Quelques satatistiques sur les notations :

In [12]:
df.iloc[:, df.columns.isin(["Garance", "Yannis", "Matthias", "score_humain"])].describe().round(3)

,Matthias,Yannis,Garance,score_humain
count,200.000,200.000,200.000,200.000
mean,5.210,5.255,5.680,0.538
std,3.401,3.388,3.438,0.333
min,0.000,0.000,0.000,0.000
25%,2.000,2.000,3.000,0.233
50%,5.500,5.000,6.000,0.600
75%,8.000,8.000,9.000,0.800
max,10.000,10.000,10.000,1.000


In [25]:
fig = px.box(df, x="score_humain", color_discrete_sequence=["#2a2781"])
fig.update_layout(xaxis_title="Score humain", width=800, height=450)

In [4]:
# Calcul des corrélations de Pearson entre chaque paire d'évaluateurs
corr_GM = df["Garance"].corr(df["Matthias"], method='pearson')
corr_GY = df["Garance"].corr(df["Yannis"], method='pearson')
corr_MY = df["Matthias"].corr(df["Yannis"], method='pearson')

print(f"Corrélation (de Pearson) entre Garance et Matthias : {corr_GM:.2f}")
print(f"Corrélation (de Pearson) entre Garance et Yannis : {corr_GY:.2f}")
print(f"Corrélation (de Pearson) entre Matthias et Yannis : {corr_MY:.2f}")

Corrélation (de Pearson) entre Garance et Matthias : 0.93
Corrélation (de Pearson) entre Garance et Yannis : 0.93
Corrélation (de Pearson) entre Matthias et Yannis : 0.94


Les trois évaluateurs humains (Matthias, Yannis, Garance) semblent globalement en accord.

# **III - Visualisations**

### **A - Caractéristiques des contributions**

In [5]:
# Barplot du nombre de charactères par contribution sur les 200 premières contributions
fig = px.histogram(
    df, x="nb_char_contrib", nbins=max(df["nb_char_contrib"])//100,
    color_discrete_sequence=["#2a2781"]
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre de charactères par contribution</b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Nombre de contributions", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de charactères = %{x}<br>" \
        "Nombre de contributions = %{y}<extra></extra>"
)
fig.show()

Au vu de la distribution, il semble judicieux de regrouper les contributions en différentes catégories de longueur (arbitraire).

### **B - Caractéristiques des extractions**

In [6]:
fig = px.histogram(
    df, x="contrib_cat", y="Hallucinations",
    color_discrete_sequence=["#2a2781"], histfunc="sum", category_orders=dict(contrib_cat=length_labels)
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre d'hallucinations en fonction du nombre de <br>charactères par contribution</b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Nombre d'hallucinations", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre d'hallucinations = %{y}<extra></extra>"
)
fig.show()

In [7]:
fig = px.histogram(
    df, x="contrib_cat", y="Idées_non_ind",
    color_discrete_sequence=["#2a2781"], histfunc="sum", category_orders=dict(contrib_cat=length_labels)
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre d'idées non indépendantes en fonction du <br>nombre de charactères par contribution</b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Nombre d'idées non indépendantes", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre d'idées non indépendantes = %{y}<extra></extra>"
)
fig.show()

In [8]:
fig = px.histogram(
    df, x="contrib_cat", y="score_humain",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders=dict(contrib_cat=length_labels)
)
fig.update_layout(
    title={"text": "<b>Distribution du score humain en fonction du <br>nombre de charactères par contribution</b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Score humain", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Score humain = %{y}<extra></extra>"
)
fig.show()